# 👩🏼‍💻 Hacker News Posts Analysis
According to wikipedia Hacker News is a " social news website focusing on computer science and entrepreneurship. It is run by the investment fund and startup incubator Y Combinator. In general, content that can be submitted is defined as "anything that gratifies one's intellectual curiosity.""

In this analysis we are going to look at the stories(posts) posted on the website in 2022 with a particular interest on Ask HN and Show HN posts. The main outcome of this analysis will be to see what is the best time to submit these specific posts on the platform to get the biggest engagement (number of comments).

This use case can be replicated for any other type of a post or website that provides their data publicly.

This project is inspired by Dataquest's project that analyses data from 2016. Since I wanted to make the analsys more relevant and bring something new to it I've decided to get the latest data from 2022. This data is now widely available in the form of a public dataset in Google's Analytics Hub as described in [this](https://github.com/minimaxir/get-all-hacker-news-submissions-comments) github post.

## Acquiring the data

**Step 1**
Open BigQuery and access the "bigquery-public-data.hacker_news.full" dataset from the Analytics Hub.

**Step 2**
Write a query to extract data from 2022. The table contains both comments and stories, therefore you need to create a temporary table or a subquery to calculate the number of comments per each article. A better scenario would be to have these two separate and connected by an identifier. Based on my investigation it looks like comments have the ID of the article in the parents column so that is what I used to join these two datapoints.

-- A: Create a subquery that will get the number of comments per parent(article)

WITH
  comments AS (
    SELECT parent, count(id) as no_of_comments
    FROM `bigquery-public-data.hacker_news.full`
    WHERE type = "comment" AND EXTRACT(YEAR FROM timestamp) >= 2022
    GROUP BY parent
  )
  
-- B: Get a list of parents(articles) and join them with the number of comments calculated in step 1

SELECT title, timestamp, id, comments.no_of_comments
FROM `bigquery-public-data.hacker_news.full` JOIN comments ON comments.parent = id
WHERE type = "story" AND EXTRACT(YEAR FROM timestamp) = 2022

**Step 3**
Export the data. There are multiple options to do this, however, this file was too big to be exported as csv and therefore I decided to go for the jsonl option. We will explore how to load a json in the next section.

In [74]:
import json
 
# Opening JSONL file
opened_file = open('HN_2022_JSON.json')
# Converting a JSONL file to a list of dictionaries
read_file = list(opened_file)

opened_file.close()

# Getting the header row by picking first dictionary and accessing its keys
row = json.loads(read_file[:1][0])
header = list(row.keys())

# Iterating over each row and storing values from each dictionary as a separate list in the list of lists
hn = []
for json_str in read_file:
    row = json.loads(json_str)
    row = list(row.values())
    hn.append(row)

# Result is a list of lists (essentially a table) containing following columns 
print(header)

# Example of one row of data
print(hn[5])

['title', 'timestamp', 'id', 'no_of_comments']
['Coinbase CEO tweets how they came up with Super Bowl ad, lied about it', '2022-02-21 21:41:29 UTC', '30420990', '51']


## Analyzing the data
In this part we are going to analyze our dataset. In order to get to our objective which I'll repeat is: "what is the best time to submit these specific posts on the platform to get the biggest engagement (number of comments)" we will do the following steps:

1. Filter out all posts that start with "Ask HN" or "Show HN"
2. Calculate the number of posts and comments that are created at every hour
3. Calculate what is the average number of comments per post per hour and sort them from high to low

**1.Filter out all posts that contain "Ask HN" or "Show HN"**

In [4]:
# This function takes in a dataset that contains a column of strings which we want to filter based on what is in the
# beginning of the string. Then it takes in the substring that should be at the start and the column position. Finally,
# there are two default arguments capitalise and inlcude. If capitalise is set to False we will disregard 
# capitalisation of the text in case it is true we will take into consideration whether the substring contains 
# uppercase or lowercase letters. If include is set to true it means we want to filter strings that contain the 
# selected string, if false give then it will remove all of the entries containing set substring and return that 
# dataset.
# This function returns a filtered dataset containing only rows that start with the specified substring in the given 
# column.

def filter_start_string_column(dataset, filter_string, column, capitalize = False, include = True):
    filtered_data = []
    if capitalize == False:
        filter_string = filter_string.lower()
        for row in dataset:
            lowered = row[column].lower()
            if lowered.startswith(filter_string) == include:
                filtered_data.append(row)
    else:
        for row in dataset:
            if row[column].startswith(filter_string) == include:
                filtered_data.append(row)
    return filtered_data        

In [69]:
ask_posts = filter_start_string_column(hn,'Ask HN',0)
show_posts = filter_start_string_column(hn,'Show HN',0)

# Filtering out all remaining posts by doing inverse filtering using the "include" argument
other_posts = filter_start_string_column(hn,'Ask HN',0, include = False)
other_posts = filter_start_string_column(other_posts,'Show HN',0, include = False)

In [70]:
# Printing lengths of all lists for checking that the above function processes the whole input
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

print(len(hn))

11887
4705
101560
118152


In [7]:
def calculate_average_per_column(dataset, column):
    sum = 0
    for row in dataset:
        sum += int(row[column])
    return sum/len(dataset)

In [73]:
# Looking at the overal average of comments per post type
avg_ask_comments = calculate_average_per_column(ask_posts,3)
avg_show_comments = calculate_average_per_column(show_posts,3)

In [72]:
print(avg_ask_comments)
print(avg_show_comments)

11.515773534112897
6.007226354941552


The above averages show that Hacker News is a very active social network that can provide answers when you ask since the average number of comments for ask comments is almost double the amount for Show HN posts.

**2. Calculate the number of posts and comments that are created at every hour**
Now we're going to look at the number of posts and their comments that are submitted at every hour. This can help us understand which times are the less busy which can lead to a better visbility of our posts.

In order to do that we need to treat the timestamp data we have in the dataset.

In [76]:
from datetime import datetime

# Testing out the strptime formatting
date_hour = datetime.strptime(ask_posts[0][1],"%Y-%m-%d %H:%M:%S %Z") #'2022-06-08 01:26:10 UTC'
print(ask_posts[0][1])
print(date_hour.hour)

2022-08-17 11:58:58 UTC
11


In [79]:
number_of_ask_posts = {}
number_of_ask_comments = {}
number_of_show_posts = {}
number_of_show_comments = {}

# Iterating over each row in the lists and extracting the hour when they were posted
# With each passing we augment the number of posts and we also addition the number of comments to get a 
# dictionary that will have hours of the day as keys and number of posts and comments as values
for row in ask_posts:
    post_date = datetime.strptime(row[1],"%Y-%m-%d %H:%M:%S %Z")
    post_hour = str(post_date.hour)
    if post_hour in number_of_ask_posts:
        number_of_ask_posts[post_hour] += 1
        number_of_ask_comments[post_hour] += int(row[3])
    else:
        number_of_ask_posts[post_hour] = 1
        number_of_ask_comments[post_hour] = int(row[3])

print(number_of_ask_posts)
print(number_of_ask_comments)

for row in show_posts:
    post_date = datetime.strptime(row[1],"%Y-%m-%d %H:%M:%S %Z")
    post_hour = str(post_date.hour)
    if post_hour in number_of_show_posts:
        number_of_show_posts[post_hour] += 1
        number_of_show_comments[post_hour] += int(row[3])
    else:
        number_of_show_posts[post_hour] = 1
        number_of_show_comments[post_hour] = int(row[3])

print(number_of_show_posts)
print(number_of_show_comments)

{'11': 466, '15': 790, '16': 792, '22': 497, '19': 653, '12': 533, '20': 617, '9': 395, '13': 589, '18': 695, '8': 361, '7': 316, '14': 721, '17': 770, '4': 351, '21': 587, '3': 276, '10': 433, '5': 295, '1': 346, '2': 335, '23': 404, '6': 287, '0': 378}
{'11': 7201, '15': 16784, '16': 12212, '22': 4640, '19': 5727, '12': 7645, '20': 5233, '9': 4546, '13': 7526, '18': 6525, '8': 4873, '7': 3429, '14': 8613, '17': 7626, '4': 3347, '21': 5112, '3': 2414, '10': 5504, '5': 2720, '1': 3523, '2': 3540, '23': 2668, '6': 2842, '0': 2638}
{'4': 76, '6': 106, '18': 306, '8': 120, '15': 379, '13': 316, '14': 357, '9': 142, '16': 388, '12': 274, '22': 161, '19': 265, '21': 208, '2': 83, '10': 174, '17': 340, '11': 211, '0': 87, '7': 124, '20': 220, '3': 81, '1': 85, '23': 122, '5': 80}
{'4': 669, '6': 613, '18': 2112, '8': 691, '15': 2243, '13': 2561, '14': 2112, '9': 1123, '16': 2394, '12': 1867, '22': 791, '19': 1530, '21': 861, '2': 387, '10': 895, '17': 1980, '11': 1014, '0': 418, '7': 525, '2

**3. Calculate what is the average number of comments per post per hour and sort them from high to low**
As our final step we are going to take the two dictionaries created previously and we'll get the average number of comments per each hour.

In [81]:
ask_posts_per_hour = []
show_posts_per_hour = []

for key in number_of_ask_comments:
    ask_posts_per_hour.append([key, number_of_ask_comments[key]/number_of_ask_posts[key]])
    
for key in number_of_show_comments:
    show_posts_per_hour.append([key, number_of_show_comments[key]/number_of_show_posts[key]])

In [82]:
# Average number of comments per hour of ask posts
print(ask_posts_per_hour)
print(show_posts_per_hour)

[['11', 15.452789699570815], ['15', 21.245569620253164], ['16', 15.419191919191919], ['22', 9.336016096579478], ['19', 8.770290964777947], ['12', 14.343339587242026], ['20', 8.481361426256077], ['9', 11.50886075949367], ['13', 12.777589134125636], ['18', 9.388489208633093], ['8', 13.498614958448753], ['7', 10.85126582278481], ['14', 11.945908460471568], ['17', 9.903896103896104], ['4', 9.535612535612536], ['21', 8.708688245315162], ['3', 8.746376811594203], ['10', 12.711316397228638], ['5', 9.220338983050848], ['1', 10.18208092485549], ['2', 10.567164179104477], ['23', 6.603960396039604], ['6', 9.902439024390244], ['0', 6.978835978835979]]
[['4', 8.802631578947368], ['6', 5.783018867924528], ['18', 6.901960784313726], ['8', 5.758333333333334], ['15', 5.91820580474934], ['13', 8.104430379746836], ['14', 5.915966386554622], ['9', 7.908450704225352], ['16', 6.170103092783505], ['12', 6.813868613138686], ['22', 4.913043478260869], ['19', 5.773584905660377], ['21', 4.139423076923077], ['2',

Now we'll just format and sort the output.

In [83]:
# Sorting the hours by the average number of comments and keeping only top 5
from operator import itemgetter 

sorted_ask_list = sorted(ask_posts_per_hour, key=itemgetter(1), reverse=True)
print(sorted_ask_list[:5])

sorted_show_list = sorted(show_posts_per_hour, key=itemgetter(1), reverse=True)
print(sorted_show_list[:5])
        

[['15', 21.245569620253164], ['11', 15.452789699570815], ['16', 15.419191919191919], ['12', 14.343339587242026], ['8', 13.498614958448753]]
[['23', 10.688524590163935], ['4', 8.802631578947368], ['13', 8.104430379746836], ['9', 7.908450704225352], ['18', 6.901960784313726]]


In [142]:
for row in sorted_ask_list[:5]:
    text = "At {:02d}:00 the average number of comments was: {:.2f}".format(int(row[0]), row[1])
    print(text)

At 15:00 the average number of comments was: 21.25
At 11:00 the average number of comments was: 15.45
At 16:00 the average number of comments was: 15.42
At 12:00 the average number of comments was: 14.34
At 08:00 the average number of comments was: 13.50


Based on the above, it looks like if you want to get more answers to your "Ask HN" posts they should be posted at 3pm UTC.

In [141]:
for row in sorted_show_list[:5]:
    text = "At {:02d}:00 the average number of comments was: {:.2f}".format(int(row[0]), row[1])
    print(text)

At 23:00 the average number of comments was: 10.69
At 04:00 the average number of comments was: 8.80
At 13:00 the average number of comments was: 8.10
At 09:00 the average number of comments was: 7.91
At 18:00 the average number of comments was: 6.90


Surprisingly, for "Show HN" posts the trend is completely different. The best time to post for best engagement is at 11pm UTC. Now this can have many explications that we won't be able to get from the dataset but my main hypothesis is:

**The distribution of users time zone.** The data is collected using UTC which is the universal time based on Greenwhich time meaning that it is very different to US time zones for example. Hacker News is based in [New York](https://www.crunchbase.com/organization/the-hacker-news) which might suggest(or it might not) that a large set of users are from US and therefore 23:00 UTC will be a much more reasonable evening time in most of US. 

Let's convert the time zone to EDT just to show how one might look up at which time he should publish their posts in their local time zone. 
EDT (Eastern Daylight time) is 4 hours behind UTC, also formerly known as GMT. For easy conversion you can visit [this](https://savvytime.com/converter/utc-to-est) site.

In [161]:
# This function converts given time from a selected time zone to another one. The source of the abbreviations used
# comes from https://www.timeanddate.com/time/zones/.

def convert_time(hour, from_zone, to_zone, minute = 0, file = "UTC.csv"):
    from csv import reader
    with open(file) as file:
        read_file = reader(file)
        time_zones_UTC = list(read_file)
        time_zones_UTC = time_zones_UTC[1:] # removing header
        
    # In case the user uploads a file with lower case timezones
    from_zone = from_zone.upper()
    to_zone = to_zone.upper()
        
    UTC_dict = {}
    for row in time_zones_UTC:
        zone = row[0].upper()
        hours = row[1]
        minutes = row[2]
        UTC_dict[zone] = [hours, minutes]
    
    # Checks input for validity i.e. the zone is on the list
    if (from_zone not in UTC_dict) or (to_zone not in UTC_dict):
        return("Sorry this time zone isn't on the list.")

    minute_UTC = int(minute) - int(UTC_dict[from_zone][1]) + int(UTC_dict[to_zone][1])

    overflow_hour = minute_UTC/60

    if overflow_hour >= 1:
        # Using a modulo in case there is an overflow to another day
        hour_UTC = (hour + floor(overflow_hour) - int(UTC_dict[from_zone][0]) + int(UTC_dict[to_zone][0]))%24

    else:
        hour_UTC = (hour - int(UTC_dict[from_zone][0]) + int(UTC_dict[to_zone][0]))%24
    return('{:02d}:{:02d}'.format(hour_UTC, minute_UTC))


Now let's try the new function on the two lists we've created previously. As mentioned above we are converting UTC to EDT.

Ask posts first:

In [167]:
for row in sorted_ask_list[:5]:
    print("At", convert_time(int(row[0]), "UTC", "EDT"), "the average number of comments was: {:.2f}".format(row[1]))

At 11:00 the average number of comments was: 21.25
At 07:00 the average number of comments was: 15.45
At 12:00 the average number of comments was: 15.42
At 08:00 the average number of comments was: 14.34
At 04:00 the average number of comments was: 13.50


Finally, let's look at show posts:

In [168]:
for row in sorted_show_list[:5]:
    print("At", convert_time(int(row[0]), "UTC", "EDT"), "the average number of comments was: {:.2f}".format(row[1]))

At 19:00 the average number of comments was: 10.69
At 00:00 the average number of comments was: 8.80
At 09:00 the average number of comments was: 8.10
At 05:00 the average number of comments was: 7.91
At 14:00 the average number of comments was: 6.90


The above shows us that if you're living in the US, the best time to post ask posts would be in the morning hours whereas for show posts it is distributed more evenly during the day. This could mean that comments on ask posts are coming from users in timezones that are more similar. On the other hand the more even distribution of show points could mean that people either read the posts throughtout the day or each of the peaks corresponds to a different timezone. An analysis of the user base could reveal that.